In [1]:
%%time

from IPython.display import clear_output
clear_output()

# 저니크 모멘트 계산 

from AZernike import *

# 차수에 따른 연산 속도 

print_curr_time()

line = "*"*60
line2 = "\n" + line 

def test_zernike_moments( datas, use_gpu, Ks, Ts, **options ) : 
    
    if is_array( use_gpu ) :
        print( "Hello CG ......") 
    
        for use_gpu in use_gpu :
            options = test_zernike_moments( datas, use_gpu, Ks, Ts, **options )
        pass  
    
        return options  
    else :
        print( "Hello Real ......") 
        
        if not options : 
            print( "Options are created." )
            options = {}
            options[ "debug" ] = 0 
            options[ "use_gpu" ] = use_gpu
            options[ "hash" ] = {}
            options[ "use_hash" ] = 0 
            options[ "use_thread"] = 0 
        pass 
        
        print( "Options = ", options )
    
        device = "GPU" if options["use_gpu"] else "CPU"
        multi = "Multi " if options["use_thread"] else ""
        
        key = f"{multi}{device}"
        print( f"key = {key}")
        
        if is_scalar( Ks ) :
            Ks = [ Ks ]
        pass
        
        if not key in datas :             
            data = {}
            data[ "Ks"] = Ks
            data[ "run_times" ] = []
            datas[ key ] = data 
        pass
    
        data = datas[ key ]
        
        run_times = data[ "run_times" ]
        
        print( line2 )
        print( f"use_gpu = { options['use_gpu'] }, use_hash={ options['use_hash'] }" )

        for K in Ks : 
            if not is_array( Ts ) :
                Ts = [ Ts ]
            pass
        
            for T in Ts : 
                print( line2 )
                print( "rho theta")

                circle_type = "outer" 

                rho, theta, x, y, dx, dy, k = rho_theta( 1000*K, circle_type, **options ) 

                print( f"rho shape = {rho.shape}" )

                img = cv.imread( 'image/lenna.png', 0 )

                print( "img shape= ", img.shape )

                img_org = img 

                np = cupy if options['use_gpu'] else numpy

                img = cv.resize( img_org, (int(K*1_000), int(K*1_000)), interpolation=cv.INTER_AREA )

                if options['use_gpu']  : 
                    img = np.array( img )
                pass

                img_org = img

                print( "img shape= ", img.shape )

                print( line ) 
                
                print( f"K = {K}, T = {T}" )

                moments, run_time = calc_moments(T, img, rho, theta, dx, dy, **options )
                
                run_times.append( run_time )
            pass # T
        pass # K
    
        return options
    pass
pass
    
Ks = numpy.arange( 2, 3.1, 0.5 ) 

use_gpu =  [0, 1]

T_MAX = 25 

max_mem = max_gpu_memory() if False else max_cpu_memory() 

if max_mem < 8 :
    T_MAX = 18
elif max_mem < 25 :
    T_MAX = 20
pass

T_MAX = 5 

T = T_MAX 

datas = { }

test_zernike_moments( datas, use_gpu, Ks, T ) 

print( "\nPlotting .... ")
    
# 서브 챠트 생성 
row_cnt = 1
col_cnt = 1

fig, charts = plt.subplots( row_cnt, col_cnt, figsize=(8*col_cnt, 6*row_cnt) )
charts = charts.ravel() if row_cnt*col_cnt > 1 else [charts]
chart_idx = 0 
chart = charts[ chart_idx ] ; chart_idx += 1
chart.set_title( f"Run-time(T={T})" ) 

for key in datas : 
    data = datas[ key ]
    
    Ks = data[ "Ks" ]
    run_times = data[ "run_times" ]
    
    x = Ks
    y = numpy.log10( run_times ) 
    
    label = f"Run-time {key}"    
    chart.plot( x, y, marker="s", label=label )
    chart.set_xlabel( "\nAxial Grid Count" )
    chart.set_ylabel( "log10(sec.s)")
    chart.set_xticks( x )
    chart.set_xticklabels( [ f"{k} K" for k in x ] )
    chart.set_xlim( numpy.min(x) -1, numpy.max(x) + 1 )

    chart.legend()
pass

plt.tight_layout(); plt.show()

print( "\nDone." )


Hello... Good morning!
Importing python packages was done.
Zernike functions are defined.
Current Time = 18:18:43
Current Time = 18:18:43
Hello CG ......
Hello Real ......
Options are created.
Options =  {'debug': 0, 'use_gpu': 0, 'hash': {}, 'use_hash': 0, 'use_thread': 0}
key = CPU

************************************************************
use_gpu = 0, use_hash=0

************************************************************
rho theta
rho shape = (4000000,)
img shape=  (512, 512)
img shape=  (2000, 2000)
************************************************************
K = 2.0, T = 20


KeyboardInterrupt: 